In [1]:
#import all libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
#load data 
world_cup = pd.read_csv('datasets/World Cup 2023 Dataset.csv')
results = pd.read_csv('datasets/results.csv')

In [3]:
world_cup.head()

,Team,Group,Previous Appearances,Previous Titles,Previous Finals,Previous Semi Finals,Current Rank
0,India,A,12,2,3,7,1
1,Pakistan,A,12,1,2,6,2
2,Australia,A,12,5,6,8,3
3,South Africa,A,7,0,0,4,4
4,England,A,12,1,4,6,5


In [4]:
results.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,08-Jan-15,Afghanistan,Scotland,Afghanistan,8 wickets,ICCA Dubai
1,10-Jan-15,Afghanistan,Ireland,Ireland,3 wickets,Dubai (DSC)
2,11-Jan-15,New Zealand,Sri Lanka,New Zealand,3 wickets,Christchurch
3,12-Jan-15,Ireland,Scotland,Ireland,3 wickets,Dubai (DSC)
4,14-Jan-15,Afghanistan,Scotland,Scotland,150 runs,Abu Dhabi


In [5]:
df = results[(results['Team_1'] == 'India') | (results['Team_2'] == 'India')]
india = df.iloc[:]
india.head()

,date,Team_1,Team_2,Winner,Margin,Ground
10,18-Jan-15,Australia,India,Australia,4 wickets,Melbourne
14,20-Jan-15,England,India,England,9 wickets,Brisbane
20,26-Jan-15,Australia,India,no result,NaN,Sydney
23,30-Jan-15,England,India,England,3 wickets,Perth
30,15-Feb-15,India,Pakistan,India,76 runs,Adelaide


In [6]:
#creating a column for matches played in 2010
year = []
for row in india['date']:
    year.append(int(row[7:]))
india ['match_year']= year
india_2010 = india[india.match_year >= 10]
india_2010.count()

date          173
Team_1        173
Team_2        173
Winner        173
Margin        165
Ground        173
match_year    173
dtype: int64

In [7]:
#narrowing to team patcipating in the world cup
worldcup_teams = ['England', ' South Africa', '', 'West Indies', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India']
df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()
df_teams.count()

date      1298
Team_1    1298
Team_2    1298
Winner    1298
Margin    1232
Ground    1298
dtype: int64

In [8]:
df_teams.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,08-Jan-15,Afghanistan,Scotland,Afghanistan,8 wickets,ICCA Dubai
1,10-Jan-15,Afghanistan,Ireland,Ireland,3 wickets,Dubai (DSC)
2,11-Jan-15,New Zealand,Sri Lanka,New Zealand,3 wickets,Christchurch
4,14-Jan-15,Afghanistan,Scotland,Scotland,150 runs,Abu Dhabi
5,15-Jan-15,New Zealand,Sri Lanka,Sri Lanka,6 wickets,Hamilton


In [9]:
#dropping columns that wll not affect match outcomes
df_teams_2010 = df_teams.drop(['date','Margin', 'Ground'], axis=1)
df_teams_2010.head()

,Team_1,Team_2,Winner
0,Afghanistan,Scotland,Afghanistan
1,Afghanistan,Ireland,Ireland
2,New Zealand,Sri Lanka,New Zealand
4,Afghanistan,Scotland,Scotland
5,New Zealand,Sri Lanka,Sri Lanka


In [10]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2010 = df_teams_2010.reset_index(drop=True)
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_1,'winning_team']=1
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team_2, 'winning_team']=2
df_teams_2010 = df_teams_2010.drop(['winning_team'], axis=1)

df_teams_2010.head()

,Team_1,Team_2,Winner
0,Afghanistan,Scotland,Afghanistan
1,Afghanistan,Ireland,Ireland
2,New Zealand,Sri Lanka,New Zealand
3,Afghanistan,Scotland,Scotland
4,New Zealand,Sri Lanka,Sri Lanka


In [11]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_2010, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [12]:
final.head()

,Winner,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Nepal,Team_1_Netherlands,...,Team_2_Nepal,Team_2_Netherlands,Team_2_New Zealand,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_West Indies,Team_2_Zimbabwe
0,Afghanistan,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,Ireland,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,New Zealand,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,Scotland,True,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,Sri Lanka,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [13]:

rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.722
Test set accuracy:  0.685


In [14]:
#adding ICC rankings
#the team which is positioned higher on the ICC Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

# Loading new datasets
ranking = pd.read_csv('datasets/icc_rankings.csv') 
fixtures = pd.read_csv('datasets/fixtures.csv')

# List for storing the group stage games
pred_set = []

In [15]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
40,1,6.0,7.0,09-11-2023,Bengaluru,New Zealand,Sri Lanka,Group A,NaN
41,1,4.0,9.0,10-11-2023,Ahmedabad,South Africa,Afghanistan,Group A,NaN
42,1,1.0,14.0,11-11-2023,Bengaluru,India,Netherlands,Group A,NaN
43,1,3.0,8.0,12-11-2023,Pune,Australia,Bangladesh,Group A,NaN
44,1,5.0,2.0,12-11-2023,Kolkata,England,Pakistan,Group A,NaN


In [16]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,England,New Zealand,None
1,Pakistan,Netherlands,None
2,Bangladesh,Afghanistan,None
3,South Africa,Sri Lanka,None
4,India,Australia,None


In [17]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_Hong Kong,Team_1_India,Team_1_Ireland,Team_1_Nepal,Team_1_Netherlands,Team_1_New Zealand,...,Team_2_Nepal,Team_2_Netherlands,Team_2_New Zealand,Team_2_Pakistan,Team_2_Scotland,Team_2_South Africa,Team_2_Sri Lanka,Team_2_U.A.E.,Team_2_West Indies,Team_2_Zimbabwe
0,False,False,False,True,0,False,0,0,0,False,...,0,False,True,False,0,False,False,0,0,0
1,False,False,False,False,0,False,0,0,0,False,...,0,True,False,False,0,False,False,0,0,0
2,False,False,True,False,0,False,0,0,0,False,...,0,False,False,False,0,False,False,0,0,0
3,False,False,False,False,0,False,0,0,0,False,...,0,False,False,False,0,False,True,0,0,0
4,False,False,False,False,0,True,0,0,0,False,...,0,False,False,False,0,False,False,0,0,0


In [18]:
#group matches 
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

New Zealand and England
Winner: England

Netherlands and Pakistan
Winner: Pakistan

Afghanistan and Bangladesh
Winner: Bangladesh

Sri Lanka and South Africa
Winner: South Africa

Australia and India
Winner: India

Netherlands and New Zealand
Winner: New Zealand

Bangladesh and England
Winner: England

Afghanistan and India
Winner: India

Sri Lanka and Pakistan
Winner: Pakistan

South Africa and Australia
Winner: Australia

Bangladesh and New Zealand
Winner: New Zealand

Afghanistan and England
Winner: England

Pakistan and India
Winner: India

Sri Lanka and Australia
Winner: Australia

Netherlands and South Africa
Winner: South Africa

Afghanistan and New Zealand
Winner: New Zealand

Bangladesh and India
Winner: India

Australia and Pakistan
Winner: Pakistan

Netherlands and Sri Lanka
Winner: Sri Lanka

England and South Africa
Winner: South Africa

New Zealand and India
Winner: India

Afghanistan and Pakistan
Winner: Pakistan

Bangladesh and South Africa
Winner: South Africa

Netherl

In [19]:
# List of tuples before 
semi = [('New Zealand', 'India'),
            ('England', 'South Africa')]

In [20]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [21]:
clean_and_predict(semi, ranking, final, rf)

New Zealand and India
Winner: India

England and South Africa
Winner: South Africa



In [22]:
# Finals
finals = [('India', 'England')]

In [23]:
clean_and_predict(finals, ranking, final, rf)

England and India
Winner: India

